In [ ]:
import pandas as pd
from  tqdm.notebook import tqdm

# Eksploracja danych nypd-motor-vehicle-collisions.csv

## Wczytanie danch
Wczytanie danych o wypadkach i kolizjach w mieście Nowy Jork z pliku csv.
Plik posiada nagłówek z nazwami kolumn a jako separatora użyto znaku ','

In [ ]:
# Wczytuję dana z pliku csv
ny_collisions = pd.read_csv("data/nypd-motor-vehicle-collisions.csv")

In [ ]:
ny_collisions.head(3)

In [ ]:
# Z powodu komunikatu Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
# przyglądam się kolumnie (3) "ZIP CODE"
ny_collisions["ZIP CODE"]

Pandas potraktował wartości kolumny ZIP CODE jako liczby. Bezpośrednio w pliku CSV jest to głównie 5 cyfr, ponadto kolumna posiada wartości puste lub wypełnione białymi znakami.
Traktuję kolumnę **ZIP CODE** jako ciąg znaków (typ str) a kolumnę **ACCIDENT DATE** jako dane typu datetime i wczytuję dane ponownie.

In [ ]:
# kolumna ZIP CODE jako str, a ACCIDENT DATE jako datetime
ny_collisions = pd.read_csv("data/nypd-motor-vehicle-collisions.csv",dtype={"ZIP CODE":'str'}, parse_dates=["ACCIDENT DATE"])
ny_collisions.head(3)

## Podstawowe informacje o wczytanym DataFrame
Wczytany DataFrame ny_collisions posiada 1 612 178 wierszy w 29 kolumnach.

Dane o kolumnach DataFrame ny_collisions:
* **ACCIDENT DATE**: Data wypadku, podczas wczytywania kolumna przekształcona na typ danych 'datetime64[ns]'
* **ZIP CODE**: Kod pocztowy, typ danych ciąg znaków 'str'

In [ ]:
# Podstawowe informacje o DataFrame ny_colision
# 1612178 wierszy i 29 kolumn
ny_collisions.shape

In [ ]:
ny_collisions.describe

In [ ]:
ny_collisions.info

In [ ]:
# Nazwy kolumn DataFrame ny_colision
ny_collisions.columns

In [ ]:
# Typy danych kolumn
print("ACCIDENT DATE: ",ny_collisions["ACCIDENT DATE"].dtype)
print("ZIP CODE: ",ny_collisions["ZIP CODE"].dtype)
print("COLLISION_ID: ",ny_collisions["COLLISION_ID"].dtype)

## Exploracja danych w poszczególnych kolumnach
### Kolumna COLLISION_ID
Sprawdzam czy kolumnę COLLISION_ID można użyć jako klucza głównego DataFrame, jednoznacznie wskazującego wiersz

In [ ]:
# typ danych
ny_collisions["COLLISION_ID"].dtype

In [ ]:
# czy są komórki nie uzupełnione
ny_collisions["COLLISION_ID"].isna().sum()

In [ ]:
# Czy wartości w kolumnie są unikalne ?
cnt_no_uniq = (ny_collisions["COLLISION_ID"].value_counts() > 1).sum()
cnt_no_uniq
# Niestety nie, jest jak poniżej pewna liczba zduplikowanych wartości w kolumnie COLLISION_ID

In [ ]:
# rozpiętość ilości nie unikalnych wartości w kolumnie COLLISION_ID
ny_collisions["COLLISION_ID"].value_counts().agg(["min","max"])
# Kolumna posiada maksymalnie dwie powielone wartości w COLLISION_ID.

Wyświetlam wiersze DataFrame z powielonymi wartościami w kolumnie COLLISION_ID

In [ ]:
coll_id_s = ny_collisions["COLLISION_ID"].value_counts() > 1
coll_id_s[coll_id_s].index
# dataframe ny_collisions z powielonymi wartościami w kolumnie COLLISION_ID

In [ ]:
# Wyświetlam jeden z wierzy aby przyjżeć się kolumnom 
ny_collisions.loc[ny_collisions["COLLISION_ID"] == 3126615]

Z 2 wierszy wyświetlonych powyżej wygląda że kolumny prawdopodobnie mają te same wartości w wszystkich dwóch wierszach i będzie można wyczyścić dane w DataFrame Korzystając z funkcji drop_duplicates(). Najpierw trochę zabawy z funkcją sprawdzającą czy kolumny w dataframe z zduplikowanymi wartościami COLLISION_ID, posiadją te same wartości.

In [ ]:
# "COLLISION_ID"
# Funkcja grupuje DataFrame df po kolumnie col, wyszukuje grupy które zawierają więcej niż 1 wiersz i
# sprawdza czy wartości w zgrupowanych kolumnach są różne.
# Funkcja zwraca słownik gdzie klucz to wartość grupowanej kolumny, a wrtość to lista kolumn, które się różnią
# W przypadku gdy funkcja zwróci pusty słownik, dataframe df albo nie posiada zduplikowanych wierszy albo wszystkie kolumny
# w zduplikowanych wierszach posiadają identyczne wartości
def no_unique_columns(df,col):
    no_uniq = {} # {zduplikowana_wartość:[nazwa_kolumy_1_z_róznymi_wartościami,nazwa_kolumy_2_z_róznymi_wartościami,...}
    for g in tqdm(df.groupby(col)):
        if len(g[1]) > 1:
            # Jeżeli są duplikaty badamy wartości w odpowiednich kolumnach
            for c in g[1].columns:
                if len(g[1][c].unique()) > 1:
                    # wartość zduplikowana (g[0]), posiada rózne wartości w dopowiednich kolumnach. Dodajemy informacje do słownika
                    if g[0] in no_uniq:
                        no_uniq[g[0]].append(c)
                    else:
                        no_uniq[g[0]] = [c]
    return no_uniq

In [ ]:
# test unikalności wartości. Chwilkę może potrwać ... (4 min.)
nuq = no_unique_columns(ny_collisions,"COLLISION_ID")
nuq

In [ ]:
# Wszystkie wiersze z zdupliownymi wartościami COLLISION_ID mają również zduplikowane wartości w wszystkich kolumnach.
# Czyszczę DataFrame
cnt_before = len(ny_collisions)
ny_collisions = ny_collisions.drop_duplicates("COLLISION_ID")
cnt_after = len(ny_collisions)
print(f"Ilość wierszy przed czyszczeniem: {cnt_before}\nIlość wierszy po wyczyszczeniu: {cnt_after}\nIlość wierszy usuniętych: {cnt_before-cnt_after}")
print(f"Wierszy z nieunikalnymi wartościami w kolumnie COLLISION_ID było {cnt_no_uniq}")

In [ ]:
# Liczba zduplikowanych wartości w kolumnie COLLISION_ID
(ny_collisions["COLLISION_ID"].value_counts() > 1).sum()
# Kolumny można użyć jako klucza głównego, jednoznacznie identyfikującego wiersze DataFrame

### Kolumna "ZIP CODE"

[Kod pocztowy USA](https://en.wikipedia.org/wiki/ZIP_Code) w podstawowej formie składa się z 5 cyfr, ale może posiadać więcej niż 5 znaków.
Ponadto kod pocztowy stanu NY rozpoczyna się od znaku '1'.

In [ ]:
# Wyczyszczenie kolumny z początkowych i końcowych białych znków
ny_collisions.loc[:,"ZIP CODE"] = ny_collisions["ZIP CODE"].str.strip()

Liczba wierszy w których kolumna "ZIP CODE" posiada wartość nieuzupełnioną wynosi:

In [ ]:
# liczba wierszy z kolumną ZIP CODE równym NaN lub pystym ciągiem znanków ''
# Jeżeli kolumna ZIP CODE w pliku csv jest w postaci ,, to ma wartość NaN, jeżeli ,\s+, (,jedna lub więcej znaków białych,) ma wartość ''
count_zip_code_nan = len(ny_collisions.loc[ny_collisions["ZIP CODE"].isna() | (ny_collisions["ZIP CODE"] == '')])
count_zip_code_nan

Zgodnie z informacją o kodach pocztowych w USA, kdoy NYC powinny rozpoczynać się od znaku cyfry jeden ('1').
Liczba kodów pocztowych w kolumnie "ZIP CODE" w formacie podstawowym (5 znaków) i rozpoczynająca się od znaku cyfry '1' wynosi:

In [ ]:
# Liczba wierszy w której kolumna ZIP CODE składa się z 5 cyfr, przyczym pierwsza rozpoczyna się od znaku '1'
count_zip_code_basic = ny_collisions["ZIP CODE"].str.count("1\d{4}").sum()
count_zip_code_basic

Suma ilości wierszy w której kolumna ZIP CODE ma wartość NaN i poprawny ZIP CODE stanu NY rozpoczynający się od
znaku '1', składjący się z 5 cyfr, powinna być równa ilości wierszy DataFrame ny_collision

In [ ]:
len(ny_collisions) - count_zip_code_nan - count_zip_code_basic

Operacje poniżej normalizują kolumnę ZIP CODE, tylko i wyłącznie do typu str, dla późniejszych analiz. Pozbywam się wartości NaN przez zamianę jej na pusty ciąg znaków ''.

In [ ]:
ny_collisions.loc[ny_collisions["ZIP CODE"].isna(),"ZIP CODE"] = ''

In [ ]:
len(ny_collisions["COLLISION_ID"].loc[ny_collisions["ZIP CODE"]==''])